## Design of Parametric Multistorey House

### Model

We use the same model from Workshop 8, and add a second floor


### Code

The code for doors, windows, walls, stairs an roof is a slight modified version of the code of the related workshop.

#### Common
Contains common code snippets (e.g. imports) and utility functions

In [1]:
import math
from pyplasm import *
from larlib import *

# utility functions

"""
    Wrapper to COLOR with RGBA 0-255 scale
    
    Args:
        r:  the value of the red channel. An 
            integer between 0 and 255.
        b:  the value of the blue channel. An 
            integer between 0 and 255.
        g:  the value of the green channel. An 
            integer between 0 and 255.
        a:  the value of the alpha channel. A
            float between 0 and 1.
            Default value: 1
    
    Returns
        The COLOR function with the desired color
"""
rgba = lambda r, g, b, a=1: COLOR([r / 255.0, g / 255.0, b /255.0, a])


"""
    Easy texture application
    
    Args:
        pol: the HPC model to which apply the texture
        txt: the texture to apply (path to an image file). If txt is None,
             or gives a false boolean value, then no texture is applied
    Returns:
        An HPC model with the desired texture applied, or the HPC model
        passed as input if no texture is specified
"""
apply_texture = lambda pol, txt: TEXTURE(txt)(pol) if txt else pol

Evaluating fenvs.py..
...fenvs.py imported in 0.007221 seconds


/usr/local/lib/python2.7/dist-packages/larlib/larstruct.py:233: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  self.body = [item for item in data if item != None]


##### Stairs: Workshop 3
Code from Workshop 3 for a spiral staircase

In [2]:
def getStepParameters(radius, z):
    """
    Processes input parameters

    Args:
        x: area size on the x axis
        x: area size on the y axis
        x: area size on the z axis

    Returns:
        stepDim: tuple containing step dimension (x, y, z)
        rotationAngle: angle to witch rotate every step
        internalRadius: radius of the internal (void) circle. Could be 0
      """
    # steps dimension for good practice
    STEP_SIZE = (0.4, 0.8, 0.16)

    radius = radius / 2.0
    stepY = radius if STEP_SIZE[1] > radius else STEP_SIZE[1]
    internalRadius = radius - stepY

    rotationAngle = ASIN(STEP_SIZE[0] / stepY)

    return ((STEP_SIZE[0], stepY, STEP_SIZE[2]), 
            rotationAngle,
            internalRadius)


def ggpl_spiral_stairs(dx, dy, dz):
    """
    Returns an HPC value describing a set of spiral stairs contained
    in an area defined by input parameters.

    Args:
        x: area size on the x axis
        x: area size on the y axis
        x: area size on the z axis

    Returns:
        area: an HPC value describing a set of stairs contained in a
              boxed area defined by input parameters
    """
    stepDim, angle, radius = getStepParameters(min(dx, dy), dz)
    stepX, stepY, stepZ = stepDim

    stepsNumber = int(dz / stepZ)

    rise = CUBOID([1.3 * stepX, stepY, stepZ])
    step = STRUCT([T(2)(radius), rise, T(3)(stepZ)])

    rotation = 0
    steps = []

    # create stairs
    for i in range (0, stepsNumber + 1):
        steps += [R([1, 2])(rotation)(step), T(3)(stepZ)]
        rotation += angle
    
    # uncomment to make a central column
    # column = CYLINDER([radius, dz])(1000)
    # return STRUCT([column] + steps)

    return STRUCT(steps)

##### Doors and Windows: Workshop 7
Code from Workshop 7 for glass doors and windows

In [3]:
def addNeg(l, x):
    """
        Negates the positive number x and adds it to the list l.
        If the last element of l is negative, then adds -x to it,
        otherwise appends -x to l.
        
        Args:
            x:  the number to negate and append
            l:  the list to append
        
        Returns:
            N/A
    """
    if l != [] and l[-1] < 0:
        l[-1] -= x
    else:
        l.append(-x)


def glassStruct(X, Y, Z, occupancy, (glassColor, structColor)):
    """
        Creates a glass structure described by the input args.
        
        Args:
            X:         float list of lateral quotes (X-axis)
            Y:         float list of lateral quotes (Y-axis)
            Z:         float list of lateral quotes (Z-axis).
                       Only the first two values are used:
                       The first value for the glass depth,
                       the second value for the struct depth
            occupancy: a list of integer lists representing 
                       the incidence matrix of X on Y.
                       If occupancy[i][j] is 1, then the struct
                       is present, else there is glass
            glassColor: glass color
            structColor: struct color
        Returns
            An HPC value representing the desired structure
    """

    def aux(dx, dy, dz):
        """
            glassStruct auxiliary function. Creates the glass
            structure and scales it by the parameters
            Args:
                dx: scaling value (X-axis)
                dy: scaling value (Y-axis)
                dz: scaling value (Z-axis)
            Returns
                An HPC value representing the desired structure,
                scaled accordingly
        """
        prodX = lambda *args: reduce(lambda x, y: PROD([x, y]), args)
        struct = []
        glass = []
        for iY in range(0, len(Y)):
            structV = []
            glassV = []
            for iX in range(0, len(X)):
                value = X[iX]
                if occupancy[iY][iX] == 1:
                    structV.append(value)
                    addNeg(glassV, value)
                else:
                    addNeg(structV, value)
                    glassV.append(value)

            t = T([2])(Y[iY])
            if len(structV) > 1:
                struct.append(prodX(QUOTE(structV), QUOTE([Y[iY]]), QUOTE([Z[1]])))

            if len(glassV) > 1:
                glass.append(prodX(QUOTE(glassV), QUOTE([Y[iY]]), QUOTE([Z[0]])))

            struct.append(t)
            glass.append(t)

        final_struct = STRUCT(
            [
                T(3)(Z[1] / 2.0),
                rgba(*glassColor)(STRUCT(glass)),
                T(3)(-(Z[1] / 2.0)),
                rgba(*structColor)(STRUCT(struct))
            ])

        return STRUCT([S([1, 2, 3])([dx, dy, dz]), final_struct])
    return aux


def ggpl_window(X, Y, Z, occupancy):
    """
        Creates a window described by the input args.
        
        Args:
            X:         float list of lateral quotes (X-axis)
            Y:         float list of lateral quotes (Y-axis)
            Z:         float list of lateral quotes (Z-axis).
                       Only the first two values are used:
                       The first value for the glass depth,
                       the second value for the struct depth
            occupancy: a list of integer lists representing 
                       the incidence matrix of X on Y.
                       If occupancy[i][j] is 1, then the struct
                       is present, else there is glass
        Returns
            An HPC value representing a window
    """
    return glass_struct(X, Y, Z, occupancy, ((182, 208, 249), (255, 255, 255)))


def ggpl_door(X, Y, Z, occupancy):
    """
        Creates a door described by the input args.

        Args:
            X:         float list of lateral quotes (X-axis)
            Y:         float list of lateral quotes (Y-axis)
            Z:         float list of lateral quotes (Z-axis).
                       Only the first two values are used:
                       The first value for the glass depth,
                       the second value for the struct depth
            occupancy: a list of integer lists representing 
                       the incidence matrix of X on Y.
                       If occupancy[i][j] is 1, then the struct
                       is present, else there is glass
        Returns
            An HPC value representing a door
    """
    return glass_struct(X, Y, Z, occupancy, ((182, 208, 249), (0, 0, 0)))

##### Walls and floors: Workshop 8
Code from Workshop 8 for a single story house

In [4]:
def make_base(filename):
    """
    Creates an HPC from a .lines file

    Args:
        filename: the path of the .lines file
    Returns
        an HPC value created from the .lines file, scaled accordingly

    """
    p = []
    with open(filename) as f:
        for line in f.read().splitlines():
            values = map(lambda x: float(x), line.split(','))
            p.append(POLYLINE([[values[1], values[0]], [values[3], values[2]]]))

    return S([1, 2, 3])([0.015, 0.015, 0.015])(SOLIDIFY(STRUCT(p)))


def make_walls(filename, height):
    """
    Creates an HPC value representing walls from a .lines file
    
    Args:
        filename: the path of the .lines file 
        height:   the desired height of the walls
    Returns
        an HPC value representing walls created from the .lines file
    """
    return OFFSET([0,0, height])(make_base(filename))


def make_floor(filename, height, texture):
    """
    Creates an HPC value representing a floor from a .lines file
    
    Args:
        filename: the path of the .lines file 
        height:   the desired height of the floor
        texture:  the texture to apply to the HPC
    Returns
        an HPC value representing floors created from the .lines file
    """
    return apply_texture((OFFSET([0, 0, height])(make_base(filename))), texture)

##### Roof: Workshop 9
Code from Workshop 9 for a roof

In [5]:
def pairList(inputList):
    """
    Given a list L as input, returns a new list formed by couples of consecutive
    elements of L

    Example:
        input:  [e1, e2, e3, ... , eK]
        output: [[e1, e2], [e2, e3], ..., [eK - 1, eK], [eK, e1]]

    Args:
        inputList: the list to process

    Returns:
        outputList: a new list formed by couples of consecutive elements
                    of the input list
    """

    inputLen = len(inputList)
    outputList = []

    for i in range(0, inputLen - 1):
        outputList.append([inputList[i], inputList[i + 1]])

    if (inputLen > 0):
        outputList.append([inputList[inputLen - 1], inputList[0]])

    return outputList



def rotatePoints(directions, angle):
    """
    Given a list of directions L and an angle A as input, returns a list of points
    A direction is represented with a list composed of two element ([point, quadrant]), 
    where point is a list with three integers describing the point coordinates ([x, y, z]), 
    while quadrant is an integer 1 and 4 that determines the rotation for the x and y axis.
    If quadrant is zero then no rotation is applied to the first two coordinates.
    Angle represents the desired rotation angle for the z axis.

    Example:
        input:   [[[1, 1, 1], 2], [[3, 5, 7], 1], [[8, 5, 5], 4]], 0
        output:

    Args:
        directions: a list of directions
        angle:      the angle of rotation in the z axis

    Returns:
       points:      a list of points

    """

    factors = ((1, 1), (-1, 1), (-1, -1), (1, -1))

    points = []

    for point, quadrant in directions:

        if quadrant < 1 or quadrant > 4:
            fX, fY = (0, 0)
        else:
            fX, fY = factors[quadrant - 1]

        points.append(
            [
                point[0] + fX * (30 * math.cos(math.pi / 4)),
                point[1] + fY * (30 * math.sin(math.pi / 4)),
                point[2] + (30 * math.sin(angle))
            ])

    return points + [points[0]]

def ggpl_create_roof(verts, cells, angle, directions):
    """
    Given a list of verts, a list of cells, an angle and a 
    list of directions, returns an HPC model representing a roof
    
    Args:
        verts: list of verts
        cells: list of cells
        angle: rotation angle (z axis)
        directions: list of directions
    
    Returns:
        An HPC model representing a roof
    """

    # determine "bottom" verts
    bottomVerts = rotatePoints(directions, angle)

    # create slopes and "upper" roof polygon 
    bottom = pairList(bottomVerts)
    top = pairList(verts)
    
    slopes = []
    for i in range(0, len(top)):
        slopes.append(MKPOL([[
                              top[i][0], top[i][1], 
                              bottom[i][0], bottom[i][1]
                             ],
                             [[1, 2, 3, 4]],
                             1]))
        
    roof = STRUCT([MKPOL([bottomVerts, cells, 1])] + slopes + [MKPOL([verts, cells, 1])])
    
    return apply_texture(roof, 'txt/roof.jpg')

##### Multistorey House: Workshop 10
Creates a multistorey house using code from previously developed workshops.

In [6]:
house = {
    'floor_1' :
    {
        'height': 3,
        'pavement':
        [
            {
                'lines_file': 'lines/f1/ext_pavement.lines',
                'texture_file': 'lines/f1/ext.lines',
                'height': 0.05
            },
            {
                'lines_file': 'lines/f1/parquet.lines',
                'texture_file': 'txt/parquet.jpg',
                'height': 0.1
            },
            {
                'lines_file': 'lines/f1/bath_floor.lines',
                'texture_file': 'txt/parquet.jpg',
                'height': 0.1
            }
        ],
        'walls':
        [
            {
                'lines_file': 'lines/f1/ext_walls.lines',
                'texture_file': None,
                'height': 3
            },
            {
                'lines_file': 'lines/f1/int_walls.lines',
                'texture_file': None,
                'height': 3
            }
        ],
        'windows_loc':
        [
            {
                'lines_file': 'lines/f1/windows.lines',
                'texture_file': None,
                'height': 3
            }
        ],
        'doors_loc':
        [
            {
                'lines_file': 'lines/f1/doors.lines',
                'texture_file': None,
                'height': 3
            }
        ]
    },
    'floor_2' :
    {
        'height': 3,
        'pavement':
        [
            {
                'lines_file': 'lines/f2/parquet.lines',
                'texture_file': 'txt/parquet.jpg',
                'height': 0.1
            },
        ],
        'walls':
        [
            {
                'lines_file': 'lines/f2/ext_walls.lines',
                'texture_file': None,
                'height': 3
            }
        ],
        'windows_loc':
        [
            {
                'lines_file': 'lines/f2/windows.lines',
                'texture_file': None,
                'height': 3
            }
        ]
    },
    'stairs':
    {
        'lines_file': 'lines/f1/stairs.lines',
        'texture_file': None,
        'height': 3 
    },
    'roof':
    {}
}

In [7]:
def make_doors(spec):
    def make_door(x, y, z):
        door_model = [
            # X
            [0.0125, 0.5, 0.125, 0.025,
             0.125, 0.5, 0.0125],
            # Y
            [0.025, 0.3, 0.0125, 0.3,
             0.0125, 0.3, 0.0125, 1.2,
             0.025],
            # Z
            [0.01, 0.07],
            # occupancy
            [
                [1, 1, 1, 1, 1, 1, 1],
                [1, 0, 0, 1, 0, 0, 1],
                [1, 1, 1, 1, 1, 1, 1],
                [1, 0, 0, 1, 0, 0, 1],
                [1, 1, 1, 1, 1, 1, 1],
                [1, 0, 1, 1, 1, 0, 1],
                [1, 1, 1, 1, 1, 1, 1],
                [1, 0, 0, 1, 0, 0, 1],
                [1, 1, 1, 1, 1, 1, 1]
            ]
        ]

        return S([1,2,3])([x, y, z]), ggpl_door(*door_model)

    doors = []
    door_h = float(spec['height'])

    with open(spec['lines_file'], 'r') as f:
        for line in f.read().splitlines():
            x1, y1, x2, y2 = list(map(lambda x: float(x), line.split(',')))
            x, y = 0, 0

            verts += [[x1, y1, 0], [x2, y2, 0]]

            if (x1 == x2):
                y = abs(y1 - y2)
            elif (y1 == y2):
                x = abs(x1 - x2)

            if (x != 0.0 and y != 0.0):
                doors += [
                    OFFSET([0, 0, door_h * (0.001 / 6.0)])(MKPOL(verts, [[1, 2, 3, 4]], 1)),
                    T(3)(door_h * (0.001 / 6.0)),
                    make_door(x, y, door_h * (4.0 / 6.0)),
                    T(3)(door_h * (1.0 / 6.0)),
                    OFFSET([0, 0, door_h * (1.0 / 6.0)])(MKPOL(verts, [[1, 2, 3, 4]], 1))
                ]

                x, y = 0, 0
                verts = []

    return  S([1, 2, 3])([0.015, 0.015, 0.015])(STRUCT(doors))


def make_windows(spec):
    def make_window(x, y, z):
        window_model = [
            # X
            [
                0.07, 0.15, 0.015, 0.15,
                0.015, 0.15, 0.07
            ],
            # Y
            [
                0.07, 0.18, 0.015, 0.18,
                0.07, 0.18, 0.015, 0.18,
                0.07
            ],
            # Z
            [
                0.01, 0.05
            ],
            # occupancy
            [
                [1, 1, 1, 1, 1, 1, 1],
                [1, 0, 1, 0, 1, 0, 1],
                [1, 1, 1, 1, 1, 1, 1],
                [1, 0, 1, 0, 1, 0, 1],
                [1, 1, 1, 1, 1, 1, 1],
                [1, 0, 1, 0, 1, 0, 1],
                [1, 1, 1, 1, 1, 1, 1],
                [1, 0, 1, 0, 1, 0, 1],
                [1, 1, 1, 1, 1, 1, 1],
            ]
        ]

        return S([1,2,3])([x, y, z]), ggpl_window(*window_model)

    windows = []
    windows_h = float(spec['height'])
    verts = []
    with open(spec['lines_file'], 'r') as f:
        x, y = 0, 0
        
        for line in f.read().splitlines():
            x1, y1, x2, y2 = list(map(lambda x: float(x), line.split(',')))
            verts += [[x1, y1, 0], [x2, y2, 0]]

            if (x1 == x2):
                y = abs(y1 - y2)
            elif (y1 == y2):
                x = abs(x1 - x2)

            if (x != 0.0 and y != 0.0):
                windows += [
                    OFFSET([0, 0, windows_h * (2.0 / 6.0)])(MKPOL([verts, [[1, 2, 3, 4]], 1])),
                    T(3)(windows_h * (2.0 / 6.0)),
                    make_window(x, y, windows_h * (3.0 / 6.0)),
                    T(3)(windows_h * (3.0 / 6.0)),
                    OFFSET([0, 0, windows_h * (1.0 / 6.0)])(MKPOL([verts, [[1, 2, 3, 4]], 1]))
                ]

                x, y = 0, 0
                verts = []

    return  S([1, 2, 3])([0.015, 0.015, 0.015])(STRUCT(windows))


def ggpl_multistorey_house(house):

    def make_hpc(floor, key, func, *args):
        items = []
        print(args)
        for s in floor[key]:
            print(s)
            items.append(func(*[s[arg] for arg in args]))

        if len(items) > 1:
            return STRUCT(items)
        return items[0]

    dd = []

    for f in ('floor_1', 'floor_2'):

        # make pavement

        floor_pavement = make_hpc(house[f], 'pavement', make_floor,
                                     'lines_file', 'height', 'texture_file')

        # make walls

        floor_walls = make_hpc(house[f], 'walls', make_walls,
                                  'lines_file', 'height')

        # make windows

        floor_windows = make_windows(house[f]['windows_loc'][0])

        # make doors


        # make stairs


        # make roof

        dd += [floor_pavement, floor_walls, floor_windows, T(3)(house[f]['height'])]

    return STRUCT(dd)


In [8]:
VIEW(ggpl_multistorey_house(house))

('lines_file', 'height', 'texture_file')
{'texture_file': 'lines/f1/ext.lines', 'lines_file': 'lines/f1/ext_pavement.lines', 'height': 0.05}
{'texture_file': 'txt/parquet.jpg', 'lines_file': 'lines/f1/parquet.lines', 'height': 0.1}
{'texture_file': 'txt/parquet.jpg', 'lines_file': 'lines/f1/bath_floor.lines', 'height': 0.1}
('lines_file', 'height')
{'texture_file': None, 'lines_file': 'lines/f1/ext_walls.lines', 'height': 3}
{'texture_file': None, 'lines_file': 'lines/f1/int_walls.lines', 'height': 3}


NameError: global name 'glass_struct' is not defined